In [1]:
from bs4 import BeautifulSoup
import requests
import json
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, TreebankWordTokenizer, wordpunct_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.stem import WordNetLemmatizer
import clean
import stem
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer

In [14]:
BASE_URL = 'https://www.aljazeera.net'

articles = []
articlesTitles = []
articlesLinks = []
articlesDates = []
articlesContents = []

html = requests.get(BASE_URL + '/news/').text
soup = BeautifulSoup(html, "html.parser")

newsFeedContainer = soup.find('section', {'id': 'news-feed-container'})
print('News feed container found!')
for a in newsFeedContainer.find_all('a', href=True):
    articlesLinks.append(a['href'])
    articlesTitles.append(a.text)

articlesLinks = [(i, l) for i, l in enumerate(articlesLinks) if l.startswith('/news/')]
articlesTitles = [t for i, t in enumerate(articlesTitles) if i in [i for i, l in articlesLinks]]
articlesLinks = [l for i, l in articlesLinks]

print('Articles links retrieved!')
print('Articles titles retrieved!')
for b in articlesLinks:
    url = BASE_URL + b
    print('Retrieving article content from: ' + url)
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    article = ''

    articleDate = soup.find('span', {'class': 'article-dates__published'}).text

    articlesParagraphsContainer = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})

    articlesParagraphs = articlesParagraphsContainer.find_all('p', recursive=False)

    articlesDates.append(articleDate)

    for p in articlesParagraphs:
        article += p.text + '\n'

    articlesContents.append(article)
print('Articles dates retrieved!')
print('Articles Contents retrieved!')
for i in range(len(articlesTitles)):
    articles.append({
        'title': articlesTitles[i],
        'link': articlesLinks[i],
        'date': articlesDates[i],
        'content': articlesContents[i]
    })

News feed container found!
Articles links retrieved!
Articles titles retrieved!
Retrieving article content from: https://www.aljazeera.net/news/2024/4/2/%d8%a7%d9%84%d8%b3%d9%88%d8%af%d8%a7%d9%86-%d8%aa%d8%b9%d9%84%d9%82-%d8%b9%d9%85%d9%84-%d9%82%d9%86%d9%88%d8%a7%d8%aa-%d8%b9%d8%b1%d8%a8%d9%8a%d8%a9
Retrieving article content from: https://www.aljazeera.net/news/2024/4/2/%d9%84%d8%a3%d9%88%d9%84-%d9%85%d8%b1%d8%a9-%d9%85%d9%86%d8%b0-%d9%86%d9%88%d9%81%d9%85%d8%a8%d8%b1-%d8%b5%d9%88%d8%a7%d8%b1%d9%8a%d8%ae-%d9%85%d9%86-%d9%84%d8%a8%d9%86%d8%a7%d9%86
Retrieving article content from: https://www.aljazeera.net/news/2024/4/2/%d9%82%d8%aa%d9%84%d9%89-%d9%88%d8%ac%d8%b1%d8%ad%d9%89-%d8%a8%d9%87%d8%ac%d9%88%d9%85-%d8%a7%d8%b3%d8%aa%d9%87%d8%af%d9%81-%d8%a5%d9%81%d8%b7%d8%a7%d8%b1%d8%a7-%d8%ac%d9%85%d8%a7%d8%b9%d9%8a%d8%a7
Retrieving article content from: https://www.aljazeera.net/news/2024/4/2/%d8%a7%d9%84%d8%b4%d8%a8%d9%83%d8%a9-%d8%a7%d9%84%d8%b3%d9%88%d8%b1%d9%8a%d8%a9-%d9%84%d8%ad%d9%82%d

In [15]:
with open('tmp/aljazzera.json', 'w', encoding='utf-8') as json_file:
    json.dump(articles, json_file, indent=4, ensure_ascii=False)

In [4]:
load_dotenv()

username = quote_plus(os.getenv('MONGODB_USERNAME'))
password = quote_plus(os.getenv('MONGODB_PASS'))
cluster = os.getenv('MONGODB_CLUSTER_ID')

uri = "mongodb+srv://" + username + ":" + password + "@cluster." + cluster + ".mongodb.net/?retryWrites=true&w=majority"

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
db = client['nlp-lab-db']

collection = db['aljazeera-articles']

collection.insert_many(articles)

print('Articles inserted successfully')

Articles inserted successfully


In [16]:
with open('tmp/aljazzera.json', 'r', encoding='utf-8') as json_file:
    articles = json.load(json_file)
    articleExample = articles[0]

articleExample

{'title': 'السودان يعلّق عمل وسائل إعلام عربية',
 'link': '/news/2024/4/2/%d8%a7%d9%84%d8%b3%d9%88%d8%af%d8%a7%d9%86-%d8%aa%d8%b9%d9%84%d9%82-%d8%b9%d9%85%d9%84-%d9%82%d9%86%d9%88%d8%a7%d8%aa-%d8%b9%d8%b1%d8%a8%d9%8a%d8%a9',
 'date': '2/4/2024',
 'content': 'نقلت وكالة الأنباء الرسمية السودانية عن وزير الإعلام في السودان جراهام عبد القادر قوله إن البلاد علقت، اليوم الثلاثاء، عمل قنوات "العربية" و"الحدث" و"سكاي نيوز عربية"، وذلك "لعدم التزامها بالمهنية المطلوبة والشفافية، وعدم تجديد تراخيصها"، حسب تعبيره.\nونقلت الوكالة عن وزير الثقافة والإعلام أنه أصدر قرارا وزاريا قضى بإيقاف هذه القنوات في السودان.\nوأضافت أن القرار جاء "استنادا إلى موجهات ومطلوبات المهنية والشفافية في العمل الإعلامي ومصلحة المواطن السوداني وقيمه، وذلك لعدم التزامها بالشفافية والمهنية المطلوبة، كما أنها لم تجدد تراخيصها لممارسة العمل الإعلامي".\n'}

In [17]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# Paragraphs Tok
content = articleExample['content']

content = clean.remove_punctuations(content)
content = clean.remove_diacritics(content)
content = clean.normalize_arabic(content)

contentParagraphs = content.split('\n')

contentParagraphs = list(filter(None, contentParagraphs))

contentParagraphs

['نقلت وكاله الانباء الرسميه السودانيه عن وزير الاعلام في السودان جراهام عبد القادر قوله ان البلاد علقت اليوم الثلاثاء عمل قنوات العربيه والحدث وسكاي نيوز عربيه وذلك لعدم التزامها بالمهنيه المطلوبه والشفافيه وعدم تجديد تراخيصها حسب تعبيره',
 'ونقلت الوكاله عن وزير الثقافه والاعلام انه اصدر قرارا وزاريا قضي بايقاف هذه القنوات في السودان',
 'واضافت ان القرار جاء استنادا الي موجهات ومطلوبات المهنيه والشفافيه في العمل الاعلامي ومصلحه المواطن السوداني وقيمه وذلك لعدم التزامها بالشفافيه والمهنيه المطلوبه كما انها لم تجدد تراخيصها لممارسه العمل الاعلامي']

In [19]:
# Sentences Tok
for p in contentParagraphs:
    sentences = sent_tokenize(p)
    print(sentences)

['نقلت وكاله الانباء الرسميه السودانيه عن وزير الاعلام في السودان جراهام عبد القادر قوله ان البلاد علقت اليوم الثلاثاء عمل قنوات العربيه والحدث وسكاي نيوز عربيه وذلك لعدم التزامها بالمهنيه المطلوبه والشفافيه وعدم تجديد تراخيصها حسب تعبيره']
['ونقلت الوكاله عن وزير الثقافه والاعلام انه اصدر قرارا وزاريا قضي بايقاف هذه القنوات في السودان']
['واضافت ان القرار جاء استنادا الي موجهات ومطلوبات المهنيه والشفافيه في العمل الاعلامي ومصلحه المواطن السوداني وقيمه وذلك لعدم التزامها بالشفافيه والمهنيه المطلوبه كما انها لم تجدد تراخيصها لممارسه العمل الاعلامي']


In [20]:
# Word Tok
stop_words = set(stopwords.words('arabic'))
contentWords = []

for p in contentParagraphs:
    words = word_tokenize(p)
    print("original:")
    print(words)
    words = [w for w in words if not w in stop_words]
    print("without stopwords:")
    print(words)
    print("--------------------------------------")
    contentWords.append(words)

original:
['نقلت', 'وكاله', 'الانباء', 'الرسميه', 'السودانيه', 'عن', 'وزير', 'الاعلام', 'في', 'السودان', 'جراهام', 'عبد', 'القادر', 'قوله', 'ان', 'البلاد', 'علقت', 'اليوم', 'الثلاثاء', 'عمل', 'قنوات', 'العربيه', 'والحدث', 'وسكاي', 'نيوز', 'عربيه', 'وذلك', 'لعدم', 'التزامها', 'بالمهنيه', 'المطلوبه', 'والشفافيه', 'وعدم', 'تجديد', 'تراخيصها', 'حسب', 'تعبيره']
without stopwords:
['نقلت', 'وكاله', 'الانباء', 'الرسميه', 'السودانيه', 'وزير', 'الاعلام', 'السودان', 'جراهام', 'عبد', 'القادر', 'قوله', 'ان', 'البلاد', 'علقت', 'اليوم', 'الثلاثاء', 'عمل', 'قنوات', 'العربيه', 'والحدث', 'وسكاي', 'نيوز', 'عربيه', 'وذلك', 'لعدم', 'التزامها', 'بالمهنيه', 'المطلوبه', 'والشفافيه', 'وعدم', 'تجديد', 'تراخيصها', 'تعبيره']
--------------------------------------
original:
['ونقلت', 'الوكاله', 'عن', 'وزير', 'الثقافه', 'والاعلام', 'انه', 'اصدر', 'قرارا', 'وزاريا', 'قضي', 'بايقاف', 'هذه', 'القنوات', 'في', 'السودان']
without stopwords:
['ونقلت', 'الوكاله', 'وزير', 'الثقافه', 'والاعلام', 'انه', 'اصدر', 'قرارا', 'وزا

In [24]:
# Stemming
contentWordsStemmed1 = []

for pw in contentWords:
    stemmedWords = stem.light_stem(pw)
    contentWordsStemmed1.append(stemmedWords)

contentWordsStemmed1

[['نقلت',
  'وكاله',
  'انباء',
  'رسميه',
  'سودانيه',
  'وزير',
  'اعلام',
  'سود',
  'جراهام',
  'عبد',
  'قادر',
  'قوله',
  'ان',
  'بلاد',
  'علقت',
  'اليوم',
  'ثلاثاء',
  'عمل',
  'قنو',
  'عربيه',
  'حدث',
  'وسكاي',
  'نيوز',
  'عربيه',
  'وذلك',
  'لعدم',
  'تزام',
  'مهنيه',
  'مطلوبه',
  'شفافيه',
  'وعدم',
  'تجديد',
  'تراخيص',
  'تعبيره'],
 ['ونقلت',
  'وكاله',
  'وزير',
  'ثقافه',
  'اعلام',
  'انه',
  'اصدر',
  'قرارا',
  'وزار',
  'قضي',
  'بايقاف',
  'قنو',
  'سود'],
 ['واضافت',
  'ان',
  'قرار',
  'جاء',
  'استنادا',
  'الي',
  'موجه',
  'ومطلوب',
  'مهنيه',
  'شفافيه',
  'عمل',
  'اعلامي',
  'ومصلحه',
  'مواطن',
  'سودا',
  'وقيمه',
  'وذلك',
  'لعدم',
  'تزام',
  'شفافيه',
  'مهنيه',
  'مطلوبه',
  'انها',
  'تجدد',
  'تراخيص',
  'لممارسه',
  'عمل',
  'اعلامي']]

In [25]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
contentWordsLemma1 = []

for pw in contentWords:
    words = [lemmatizer.lemmatize(w) for w in pw]
    print(words)
    contentWordsLemma1.append(words)

contentWordsLemma1

['نقلت', 'وكاله', 'الانباء', 'الرسميه', 'السودانيه', 'وزير', 'الاعلام', 'السودان', 'جراهام', 'عبد', 'القادر', 'قوله', 'ان', 'البلاد', 'علقت', 'اليوم', 'الثلاثاء', 'عمل', 'قنوات', 'العربيه', 'والحدث', 'وسكاي', 'نيوز', 'عربيه', 'وذلك', 'لعدم', 'التزامها', 'بالمهنيه', 'المطلوبه', 'والشفافيه', 'وعدم', 'تجديد', 'تراخيصها', 'تعبيره']
['ونقلت', 'الوكاله', 'وزير', 'الثقافه', 'والاعلام', 'انه', 'اصدر', 'قرارا', 'وزاريا', 'قضي', 'بايقاف', 'القنوات', 'السودان']
['واضافت', 'ان', 'القرار', 'جاء', 'استنادا', 'الي', 'موجهات', 'ومطلوبات', 'المهنيه', 'والشفافيه', 'العمل', 'الاعلامي', 'ومصلحه', 'المواطن', 'السوداني', 'وقيمه', 'وذلك', 'لعدم', 'التزامها', 'بالشفافيه', 'والمهنيه', 'المطلوبه', 'انها', 'تجدد', 'تراخيصها', 'لممارسه', 'العمل', 'الاعلامي']


[['نقلت',
  'وكاله',
  'الانباء',
  'الرسميه',
  'السودانيه',
  'وزير',
  'الاعلام',
  'السودان',
  'جراهام',
  'عبد',
  'القادر',
  'قوله',
  'ان',
  'البلاد',
  'علقت',
  'اليوم',
  'الثلاثاء',
  'عمل',
  'قنوات',
  'العربيه',
  'والحدث',
  'وسكاي',
  'نيوز',
  'عربيه',
  'وذلك',
  'لعدم',
  'التزامها',
  'بالمهنيه',
  'المطلوبه',
  'والشفافيه',
  'وعدم',
  'تجديد',
  'تراخيصها',
  'تعبيره'],
 ['ونقلت',
  'الوكاله',
  'وزير',
  'الثقافه',
  'والاعلام',
  'انه',
  'اصدر',
  'قرارا',
  'وزاريا',
  'قضي',
  'بايقاف',
  'القنوات',
  'السودان'],
 ['واضافت',
  'ان',
  'القرار',
  'جاء',
  'استنادا',
  'الي',
  'موجهات',
  'ومطلوبات',
  'المهنيه',
  'والشفافيه',
  'العمل',
  'الاعلامي',
  'ومصلحه',
  'المواطن',
  'السوداني',
  'وقيمه',
  'وذلك',
  'لعدم',
  'التزامها',
  'بالشفافيه',
  'والمهنيه',
  'المطلوبه',
  'انها',
  'تجدد',
  'تراخيصها',
  'لممارسه',
  'العمل',
  'الاعلامي']]

In [26]:
#Stemming with araby
ArListem = ArabicLightStemmer()

contentWordsStemmed2 = []

for pw in contentWords:
    stemmedWords = [ArListem.light_stem(w) for w in pw]
    contentWordsStemmed2.append(stemmedWords)

contentWordsStemmed2

[['قلت',
  'ال',
  'انباء',
  'رسميه',
  'سودانيه',
  'زير',
  'اعلام',
  'سود',
  'جراهام',
  'عبد',
  'قادر',
  'قول',
  'ان',
  'بلاد',
  'علق',
  'يوم',
  'ثلاثاء',
  'عمل',
  'قنو',
  'عربيه',
  'حدث',
  'سك',
  'يوز',
  'عرب',
  'ذلك',
  'عدم',
  'تزامه',
  'مهنيه',
  'مطلوبه',
  'شفافيه',
  'عدم',
  'جديد',
  'تراخيص',
  'عبير'],
 ['قلت',
  'وكاله',
  'زير',
  'ثقافه',
  'اعلام',
  'نه',
  'صدر',
  'قرار',
  'زاري',
  'قض',
  'ايقاف',
  'قنو',
  'سود'],
 ['اضاف',
  'ان',
  'قرار',
  'جاء',
  'ستناد',
  'لي',
  'موجه',
  'مطلوب',
  'مهنيه',
  'شفافيه',
  'عمل',
  'اعلام',
  'مصلح',
  'مواطن',
  'سودان',
  'قيم',
  'ذلك',
  'عدم',
  'تزامه',
  'شفافيه',
  'مهنيه',
  'مطلوبه',
  'نه',
  'جدد',
  'تراخيص',
  'ممارس',
  'عمل',
  'اعلام']]

In [27]:
# Lemma with qalsadi
lemmer = qalsadi.lemmatizer.Lemmatizer()

contentWordsLemma2 = []

for pw in contentWords:
    words = [lemmer.lemmatize(w) for w in pw]
    contentWordsLemma2.append(words)

contentWordsLemma2

[['نقل',
  'كال',
  'الانباء',
  'الرسميه',
  'السودانيه',
  'زير',
  'الاعلام',
  'سود',
  'جراهام',
  'عبد',
  'قادر',
  'قول',
  'ان',
  'بلاد',
  'علق',
  'يوم',
  'ثلاثاء',
  'عمل',
  'قنو',
  'العربيه',
  'حدث',
  'سك',
  'نيوز',
  'عرب',
  'ذلك',
  'عدم',
  'التزام',
  'بالمهنيه',
  'المطلوبه',
  'والشفافيه',
  'عدم',
  'تجديد',
  'تراخيصها',
  'تعبير'],
 ['نقل',
  'الوكاله',
  'زير',
  'الثقافه',
  'والاعلام',
  'نهو',
  'صدر',
  'قرار',
  'وزاري',
  'قضة',
  'بايقاف',
  'قنو',
  'سود'],
 ['واضافت',
  'ان',
  'قرار',
  'جاء',
  'استناد',
  'ال',
  'موجه',
  'مطلوب',
  'المهنيه',
  'والشفافيه',
  'عمل',
  'الاعلامي',
  'مصلح',
  'مواطن',
  'سودان',
  'قيم',
  'ذلك',
  'عدم',
  'التزام',
  'بالشفافيه',
  'والمهنيه',
  'المطلوبه',
  'انها',
  'تجدد',
  'تراخيصها',
  'ممارس',
  'عمل',
  'الاعلامي']]

# refs
Motazsaad, “GitHub - motazsaad/process-arabic-text: Pre-process arabic text (remove diacritics, punctuations and repeating characters),” GitHub. https://github.com/motazsaad/process-arabic-text

T. Zerrouki, Tashaphyne, Arabic light stemmer, https://pypi.python.org/pypi/Tashaphyne/0.2

T. Zerrouki, Pyarabic, An Arabic language library for Python, https://pypi.python.org/pypi/pyarabic/, 2010

T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/